In [16]:
import "dotenv/config";

[Module: null prototype] { default: {}, "module.exports": {} }

## Helper Utilities

In [17]:
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";
import { StructuredTool } from "@langchain/core/tools";
import { Runnable } from "@langchain/core/runnables";
import { ChatBedrockConverse } from "@langchain/aws";

/**
 * Create an agent that can run a set of tools.
 */
async function createAgent({
  llm,
  tools,
  systemMessage,
}: {
  llm: ChatBedrockConverse;
  tools: StructuredTool[];
  systemMessage: string;
}): Promise<Runnable> {
  const toolNames = tools.map((tool) => tool.name).join(", ");

  let prompt = ChatPromptTemplate.fromMessages([
    [
      "system",
      `You are a helpful AI assistant, collaborating with other assistants.
      Use the provided tools to progress towards answering the question.
      If you are unable to fully answer, that's OK, another assistant with different tools
      will help where you left off. Execute what you can to make progress.

      If you have the final answer or deliverable,
      prefix your response with FINAL ANSWER so the team knows to stop.
      Otherwise, You have access to the following tools: {tool_names}.

      **IMPORTANT**
      You must NOT do any task that can't be done with only given tools. You should delegate it to other assistants.

      {system_message}`,
    ],
    new MessagesPlaceholder("messages"),
  ]);
  prompt = await prompt.partial({
    system_message: systemMessage,
    tool_names: toolNames,
  });

  return prompt.pipe(llm.bind({ tools }));
}


## Define State

In [18]:
import { BaseMessage } from "@langchain/core/messages";
import { Annotation } from "@langchain/langgraph";

// This defines the object that is passed between each node
// in the graph. We will create different nodes for each agent and tool
const AgentState = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  }),
  sender: Annotation<string>({
    reducer: (x, y) => y ?? x ?? "user",
    default: () => "user",
  }),
});


## Define tools

In [19]:
import { TavilySearch } from "@langchain/tavily";
import { tool } from "@langchain/core/tools";
import * as d3 from "d3";
import { createCanvas } from "canvas";
import { z } from "zod";
import { writeFileSync } from "node:fs";

const chartTool = tool(
  ({ data }) => {
    const width = 500;
    const height = 500;
    const margin = { top: 20, right: 30, bottom: 30, left: 40 };

    const canvas = createCanvas(width, height);
    const ctx = canvas.getContext("2d");

    const x = d3
      .scaleBand()
      .domain(data.map((d) => d.label))
      .range([margin.left, width - margin.right])
      .padding(0.1);

    const y = d3
      .scaleLinear()
      .domain([0, d3.max(data, (d) => d.value) ?? 0])
      .nice()
      .range([height - margin.bottom, margin.top]);

    const colorPalette = [
      "#e6194B",
      "#3cb44b",
      "#4363d8",
      "#f58231",
      "#911eb4",
      "#42d4f4",
      "#f032e6",
      "#bfef45",
      "#fabebe",
    ];

    data.forEach((d, idx) => {
      ctx.fillStyle = colorPalette[idx % colorPalette.length];
      ctx.fillRect(
        x(d.label) ?? 0,
        y(d.value),
        x.bandwidth(),
        height - margin.bottom - y(d.value)
      );
    });

    ctx.beginPath();
    ctx.strokeStyle = "black";
    ctx.moveTo(margin.left, height - margin.bottom);
    ctx.lineTo(width - margin.right, height - margin.bottom);
    ctx.stroke();

    ctx.textAlign = "center";
    ctx.textBaseline = "top";
    x.domain().forEach((d) => {
      const xCoord = (x(d) ?? 0) + x.bandwidth() / 2;
      ctx.fillText(d, xCoord, height - margin.bottom + 6);
    });

    ctx.beginPath();
    ctx.moveTo(margin.left, height - margin.top);
    ctx.lineTo(margin.left, height - margin.bottom);
    ctx.stroke();

    ctx.textAlign = "right";
    ctx.textBaseline = "middle";
    const ticks = y.ticks();
    ticks.forEach((d) => {
      const yCoord = y(d); // height - margin.bottom - y(d)
      ctx.moveTo(margin.left, yCoord);
      ctx.lineTo(margin.left - 6, yCoord);
      ctx.stroke();
      ctx.fillText(d.toString(), margin.left - 8, yCoord);
    });
    Deno.jupyter.image(canvas.toBuffer());

    writeFileSync(
      "./basic_multi-agent_collaboration.png",
      new Uint8Array(canvas.toBuffer())
    );

    return "Chart has been generated and displayed to the user!";
  },
  {
    name: "generate_bar_chart",
    description:
      "Generate a bar chart from an array of data points using D3.js and displays it for the user.",
    schema: z.object({
      data: z
        .object({
          label: z.string(),
          value: z.number(),
        })
        .array(),
    }),
  }
);

const tavilyTool = new TavilySearch();


## Create graph

In [20]:
import { HumanMessage } from "@langchain/core/messages";
import type { RunnableConfig } from "@langchain/core/runnables";

// Helper function to run a node for a given agent
async function runAgentNode(props: {
  state: typeof AgentState.State;
  agent: RunnableConfig;
  name: string;
  config?: RunnableConfig;
}) {
  const { state, agent, name, config } = props;
  let result = await agent.invoke(state, config);
  // We convert the agent output into a format that is suitable
  // to append to the global state
  if (!result?.tool_calls || result.tool_calls.length === 0) {
    // If the agent is NOT calling a tool, we want it to
    // look like a human message.
    result = new HumanMessage({ ...result, name });
  }
  return {
    messages: [result],
    sender: name,
  };
}

const llm = new ChatBedrockConverse({
  model: "us.amazon.nova-pro-v1:0",
  region: process.env.BEDROCK_AWS_REGION ?? "us-east-1",
  credentials: {
    secretAccessKey: process.env.BEDROCK_AWS_SECRET_ACCESS_KEY ?? "",
    accessKeyId: process.env.BEDROCK_AWS_ACCESS_KEY_ID ?? "",
  },
});

// Research agent and node
const researchAgent = await createAgent({
  llm,
  tools: [tavilyTool, chartTool],
  systemMessage:
    `You should provide accurate data for the chart generator to use.`,
});

async function researchNode(
  state: typeof AgentState.State,
  config?: RunnableConfig
) {
  return runAgentNode({
    state,
    agent: researchAgent,
    name: "Researcher",
    config,
  });
}

// Chart Generator
const chartAgent = await createAgent({
  llm,
  tools: [chartTool],
  systemMessage: "Any charts you display will be visible by the user.",
});

async function chartNode(state: typeof AgentState.State) {
  return runAgentNode({
    state,
    agent: chartAgent,
    name: "ChartGenerator",
  });
}


In [21]:
// Example invocation
const researchResults = await researchNode({
  messages: [new HumanMessage("Research the US primaries in 2024")],
  sender: "User",
});

researchResults;


{
  messages: [
    AIMessage {
      "id": "cc020af0-0d0d-43d1-a3f3-b48c4e13d310",
      "content": [
        {
          "type": "text",
          "text": "<thinking> To research the US primaries in 2024, I will use the `tavily_search` tool to gather relevant information. I will look for information on the dates, key candidates, and significant events related to the 2024 US primaries. </thinking>\n\n"
        }
      ],
      "additional_kwargs": {},
      "response_metadata": {
        "$metadata": {
          "httpStatusCode": 200,
          "requestId": "cc020af0-0d0d-43d1-a3f3-b48c4e13d310",
          "attempts": 1,
          "totalRetryDelay": 0
        },
        "metrics": {
          "latencyMs": 2184
        },
        "stopReason": "tool_use",
        "usage": {
          "inputTokens": 1556,
          "outputTokens": 101,
          "totalTokens": 1657
        }
      },
      "tool_calls": [
        {
          "id": "tooluse_pGfnot-xRyytIkkUD8fNZg",
          "name": "tav

## Define Tool Node

In [22]:
import { ToolNode } from "@langchain/langgraph/prebuilt";

const tools = [tavilyTool, chartTool];
// THis runs tools in the graph
const toolNode = new ToolNode<typeof AgentState.State>(tools);


In [23]:
// Example invocation
await toolNode.invoke(researchResults);


{
  messages: [
    ToolMessage {
      "content": "{\n  \"query\": \"US primaries 2024 dates, key candidates, significant events\",\n  \"follow_up_questions\": null,\n  \"answer\": null,\n  \"images\": [],\n  \"results\": [\n    {\n      \"url\": \"https://www.newsweek.com/democrats-may-undo-joe-biden-change-2028-primary-2118883\",\n      \"title\": \"Democrats May Undo One of Joe Biden's Major Changes - Newsweek\",\n      \"score\": 0.32720643,\n      \"published_date\": \"Mon, 25 Aug 2025 17:19:49 GMT\",\n      \"content\": \"Although the primary is still years away, potential candidates like California Governor Gavin Newsom, former Transportation Secretary Pete Buttigieg and Representative Ro Khanna have already begun visiting states like South Carolina, New Hampshire and Iowa.\\n\\nPolitico reported that the calendar is not on the agenda, Democrats from those key states are engaging in \\\"behind-the-scenes lobbying\\\" to advocate for their states' early voting status. [...] Repr

## Define Edge Logic

In [24]:
import { AIMessage } from "@langchain/core/messages";

// Either agent can decide to end
function router(state: typeof AgentState.State) {
  const messages = state.messages;
  const lastMessage = messages[messages.length - 1] as AIMessage;
  if (lastMessage?.tool_calls && lastMessage.tool_calls.length > 0) {
    // The previous agent is invoking a tool
    return "call_tool";
  }
  if (
    typeof lastMessage.content === "string" &&
    lastMessage.content.includes("FINAL ANSWER")
  ) {
    // Any agent decided the work is done
    return "end";
  }
  return "continue";
}


## Define the Graph

In [25]:
import { END, START, StateGraph } from "@langchain/langgraph";

// 1. Create the graph
const workflow = new StateGraph(AgentState)
  // 2. Add the nodes; these will do work
  .addNode("Researcher", researchNode)
  .addNode("ChartGenerator", chartNode)
  .addNode("call_tool", toolNode);

// 3. Define the edges. We will define both reguler and conditional ones
// After a worker completes, report to supervisor
workflow.addConditionalEdges("Researcher", router, {
  // We will transition to the other agent
  continue: "ChartGenerator",
  call_tool: "call_tool",
  end: END,
});

workflow.addConditionalEdges("ChartGenerator", router, {
  // We will transition to the other agent
  continue: "Researcher",
  call_tool: "call_tool",
  end: END,
});

workflow.addConditionalEdges(
  "call_tool",
  // Each agent node updated the 'sender' field
  // the tool calling node does not, meaning
  // this edge will route back to the original agent
  // who invoked the tool
  (x) => x.sender,
  {
    Researcher: "Researcher",
    ChartGenerator: "ChartGenerator",
  }
);

workflow.addEdge(START, "Researcher");
const graph = workflow.compile();


StateGraph {
  nodes: {
    Researcher: {
      runnable: RunnableCallable {
        lc_serializable: false,
        lc_kwargs: {},
        lc_runnable: true,
        name: "Researcher",
        lc_namespace: [ "langgraph" ],
        func: [AsyncFunction: researchNode],
        tags: undefined,
        config: undefined,
        trace: false,
        recurse: true
      },
      retryPolicy: undefined,
      cachePolicy: undefined,
      metadata: undefined,
      input: {
        messages: BinaryOperatorAggregate {
          ValueType: undefined,
          UpdateType: undefined,
          lg_is_channel: true,
          lc_graph_name: "BinaryOperatorAggregate",
          value: undefined,
          operator: [Function: reducer],
          initialValueFactory: undefined
        },
        sender: BinaryOperatorAggregate {
          ValueType: undefined,
          UpdateType: undefined,
          lg_is_channel: true,
          lc_graph_name: "BinaryOperatorAggregate",
          value: "u

## Invoke

In [26]:
const streamResults = await graph.stream(
  {
    messages: [
      new HumanMessage({
        content: "Generate a bar chart of the US gdp over the past 3 years.",
      }),
    ],
  },
  { recursionLimit: 150 }
);

const prettifyOutput = (output: Record<string, any>) => {
  const keys = Object.keys(output);
  const firstItem = output[keys[0]];

  if ("messages" in firstItem && Array.isArray(firstItem.messages)) {
    const lastMessage = firstItem.messages[firstItem.messages.length - 1];
    console.dir(
      {
        type: lastMessage._getType(),
        content: lastMessage.content,
        tool_calls: lastMessage.tool_calls,
      },
      { depth: null }
    );
  }

  if ("sender" in firstItem) {
    console.log({
      sender: firstItem.sender,
    });
  }
};

for await (const output of await streamResults) {
  prettifyOutput(output);
  if (!output?.__end__) {
    console.log("----");
  }
}


{
  type: "ai",
  content: [
    {
      type: "text",
      text: "<thinking> To generate a bar chart of the US GDP over the past 3 years, I need to first gather the GDP data for the years 2020, 2021, and 2022. Once I have this data, I can use the `generate_bar_chart` tool to create the chart. </thinking>\n" +
        "\n"
    }
  ],
  tool_calls: [
    {
      id: "tooluse_nSv_Tlu2S7SucpJFKBlzEQ",
      name: "tavily_search",
      args: {
        query: "US GDP 2020, 2021, 2022",
        topic: "finance",
        timeRange: "year"
      },
      type: "tool_call"
    }
  ]
}
{ sender: "Researcher" }
----
{
  type: "tool",
  content: "{\n" +
    '  "query": "US GDP 2020, 2021, 2022",\n' +
    '  "follow_up_questions": null,\n' +
    '  "answer": null,\n' +
    '  "images": [],\n' +
    '  "results": [\n' +
    "    {\n" +
    '      "url": "https://www.forbes.com/sites/dereksaul/2024/11/01/how-the-economy-really-fared-under-bidenharris-and-trump-from-jobs-to-inflation-final-update/",